In [1]:
import numpy as np
import pandas as pd

In [7]:
ni_accidents = pd.read_csv('ni_accidents_2021_wgoogle.csv',index_col=0,low_memory=False)

In [8]:
ni_accidents.head()

   a_year  a_ref            a_District                   a_type  a_veh  a_cas  \
0    2016      1   Newry, Mourne, Down  Slight injury collision      1      1   
1    2016      2  Derry City, Strabane  Slight injury collision      2      1   
2    2016      2  Derry City, Strabane  Slight injury collision      2      1   
3    2016      2  Derry City, Strabane  Slight injury collision      2      1   
4    2016      2  Derry City, Strabane  Slight injury collision      2      1   

  a_wkday  a_day  a_month  a_hour  ...  c_move  c_school  c_pcv c_pedinj  \
0     FRI      1        1       1  ...     NaN     Other    NaN      NaN   
1     FRI      1        1       3  ...     NaN       NaN    NaN      NaN   
2     FRI      1        1       3  ...     NaN       NaN    NaN      NaN   
3     FRI      1        1       3  ...     NaN     Other    NaN      NaN   
4     FRI      1        1       3  ...     NaN     Other    NaN      NaN   

   c_vtype        lat       lon                         

In [9]:
ni_accidents.shape

(71964, 56)

In [10]:
ni_accidents.columns

Index(['a_year', 'a_ref', 'a_District', 'a_type', 'a_veh', 'a_cas', 'a_wkday',
       'a_day', 'a_month', 'a_hour', 'a_min', 'a_gd1', 'a_gd2', 'a_ctype',
       'a_speed', 'a_jdet', 'a_jcont', 'a_pedhum', 'a_pedphys', 'a_light',
       'a_weat', 'a_roadsc', 'a_speccs', 'a_chaz', 'a_scene', 'v_id', 'v_type',
       'v_tow', 'v_man', 'v_loc', 'v_junc', 'v_skid', 'v_hit', 'v_leave',
       'v_hitoff', 'v_impact', 'v_sex', 'v_agegroup', 'v_hitr', 'v_forreg',
       'c_id', 'c_class', 'c_sex', 'c_agegroup', 'c_sever', 'c_loc', 'c_move',
       'c_school', 'c_pcv', 'c_pedinj', 'c_vtype', 'lat', 'lon', 'address',
       'postcodes', 'location_infos'],
      dtype='object')

Data contains PSNI data:
- a_X data Collision Data
- v_X data Vehicle Data
- c_X data Casualty Data

Additonal data added:
- lat, lon derived from a_gd1 and a_gd2
- address and postcodes geocoded from latitude and longitude using Google Maps API
- location_infos which are location tags based on the latitude and longitude used by google maps to give additional information on the area e.g. road, residental premises, tourist attraction, public space

### Project Object: 
build a model to predict who are more likely to be involved in accidents and who is most likely injured in a car accident -- traffic injuries severity

Response Variable: 
**a_type** -- Slight injury collision / Serious injury collision / Fatal injury collision 

In [52]:
# get fist 1% data to try, first 720 rows.
accidents_720  = ni_accidents.iloc[:720]
# accidents_720

In [37]:
postcodes = accidents_720["postcodes"]

We now wanna create a new column and saves the first part of postcode, for example: BT30 of BT30 9UD, each BT~ represents an area of belfast. The whole postcode is a little specific for model, and just the first part will be easier to build a model on. 

In [49]:
accidents_720["postcodes_first4"] = accidents_720["postcodes"].apply(lambda x:x[:4])

c:\Temp\ipykernel_10676\2112690208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accidents_720["postcodes_first4"] = accidents_720["postcodes"].apply(lambda x:x[:4])


In [55]:
accidents_720.head()

,a_year,a_ref,a_District,a_type,a_veh,a_cas,a_wkday,a_day,a_month,a_hour,...,c_school,c_pcv,c_pedinj,c_vtype,lat,lon,address,postcodes,location_infos,postcodes_first4
0,2016,1,"Newry, Mourne, Down",Slight injury collision,1,1,FRI,1,1,1,...,Other,NaN,NaN,Car,54.379157,-5.685504,"87 Killyleagh Rd, Downpatrick BT30 9UD, UK",BT30 9UD,"['street_address', 'establishment', 'point_of_...",BT30
1,2016,2,"Derry City, Strabane",Slight injury collision,2,1,FRI,1,1,3,...,NaN,NaN,NaN,NaN,55.019106,-7.294136,"Foyle Bridge, Londonderry BT48, UK",BT48,"['route', 'plus_code', 'premise', 'street_addr...",BT48
2,2016,2,"Derry City, Strabane",Slight injury collision,2,1,FRI,1,1,3,...,NaN,NaN,NaN,NaN,55.019106,-7.294136,"Foyle Bridge, Londonderry BT48, UK",BT48,"['route', 'plus_code', 'premise', 'street_addr...",BT48
3,2016,2,"Derry City, Strabane",Slight injury collision,2,1,FRI,1,1,3,...,Other,NaN,NaN,Car,55.019106,-7.294136,"Foyle Bridge, Londonderry BT48, UK",BT48,"['route', 'plus_code', 'premise', 'street_addr...",BT48
4,2016,2,"Derry City, Strabane",Slight injury collision,2,1,FRI,1,1,3,...,Other,NaN,NaN,Car,55.019106,-7.294136,"Foyle Bridge, Londonderry BT48, UK",BT48,"['route', 'plus_code', 'premise', 'street_addr...",BT48
